In [3]:
import numpy as np
import pandas as pd
import datetime
import time
import re

# # 设置显示最大行数和列数
# pd.set_option('display.max_rows', None)  # 设置为None表示不限制行数
# pd.set_option('display.max_columns', None)  # 设置为None表示不限制列数
# pd.set_option('display.width', None)  # 自动检测控制台的宽度
# pd.set_option('display.max_colwidth', None)  # 设置列的最大宽度为-1，即不截断

import warnings
warnings.filterwarnings("ignore")

In [4]:
df_r_and_i = pd.read_excel(r'D:\迅雷下载\部件拆换记录_20250309.xlsx')
df_r_and_i['数据类型'] = '部件拆换记录'

In [5]:
# 使用 rename 方法修改列名
df_r_and_i = df_r_and_i.rename(columns={'拆换日期': '日期', 'ATA': '维修ATA', '拆换原因': '问题描述', '拆卸部件处理措施': '排故措施'})

In [6]:
df_r_and_i = df_r_and_i[[ '日期', '维修ATA', '问题描述', '排故措施', '运营人', '机型', '飞机序列号', '机号','数据类型', '拆卸部件件号', '拆卸部件序列号', '装上部件件号','装上部件序列号']]

In [7]:
# 定义替换规则
replace_rules = [
    (['天骄', '天骄航空', '天骄航空有限公司'], '天骄'),
    (['东航', '东方航空', '一二三航空', '一二三航空有限公司（东方航空）', '东航技术', '中国东方航空有限公司','一二三航空有限公司（东航）','一二三航空有限公司','中国东方航空集团有限公司'], '东航'),
    (['江西航', '江西航空有限公司', '江西航空'], '江西航'),
    (['南航', '南方航空', '中国南方航空有限公司','中国南方航空股份有限公司'], '南航'),
    (['国航', '中国国航', '中国国际航空股份有限公司'], '国航'),
    (['成航', '成都航空', '成都航空有限公司'], '成航'),
    (['乌航', '乌鲁木齐航空'], '乌航'),
    (['华夏', '华夏航空', '华夏航空股份有限公司'], '华夏'),
    (['金鹏', '金鹏航空'], '金鹏'),
    (['圆通', '圆通货航', '圆通航空','杭州圆通货运航空有限公司'], '圆通'),
#     (['商飞快线'], '商飞快线'),
#     (['试飞中心'], '试飞中心'),
#     (['试飞院'], '试飞院'),
#     (['商飞运营'], '商飞运营'),
    (['中飞通', '中飞通航', '中飞通用航空有限责任公司'], '中飞通'),
    (['中原龙浩', '中原龙浩航空'], '中原龙浩'),
    (['上飞公司维修中心'], '上飞公司维修中心'),
    (['GAMECO', '广州飞机维修工程有限公司（GAMECO）'], 'GAMECO'),
    (['翎亚', '翎亚航空 PT TransNusa Aviation Mandiri', 'TRANSNUSA AVIATION MANDIRI', 'TransNusa'], '翎亚'),
    (['中国商飞', '中国商用飞机有限责任公司'], '中国商飞'),
    (['AMECO', '北京飞机维修工程有限公司'], 'AMECO'),
    (['山东太古', '山东太古飞机工程有限公司'], '山东太古'),
    (['STARCO', '上海科技宇航有限公司 Shanghai Technologies Aerospace (STARCO)'], 'STARCO'),
    (['中飞租', '中国飞机租赁集团控股有限公司'], '中飞租'),
#     (['维修交付中心'], '维修交付中心'),
#     (['西安阎良试飞现场'], '西安阎良试飞现场'),
#     (['无'], '无')
]

# 过滤掉只有单个元素的规则，因为单个元素无需替换
filtered_replace_rules = [rule for rule in replace_rules if len(rule[0]) > 1]

# 进行替换操作
for replacements, target in filtered_replace_rules:
    df_r_and_i['运营人'] = df_r_and_i['运营人'].replace(replacements, target)

df_r_and_i['运营人'].fillna('无', inplace=True)
df_r_and_i['运营人'].unique()

array(['成航', '南航', '江西航', '国航', '东航', '华夏', '天骄', '翎亚', '圆通'],
      dtype=object)

In [8]:
df_faults = pd.read_parquet(r'D:\Quant\Case_Search_UI\data\raw\faults.parquet')

In [9]:
df_faults['运营人'].unique()

array(['东航', '国航', '南航', '成航', '华夏', '江西航', '天骄', '翎亚', '圆通'],
      dtype=object)

In [11]:
df_faults = pd.concat([df_faults, df_r_and_i], axis=0)

In [12]:
df_faults.sort_values('日期', ascending=False, inplace=True)
df_faults.reset_index(inplace=True, drop=True)
df_faults = df_faults.drop_duplicates()
df_faults.shape

(55773, 13)

In [33]:
# df_faults.to_parquet(r'D:\Quant\Search_UI\app\data\faults.parquet')

In [14]:
df_faults.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55773 entries, 0 to 82443
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   日期       55773 non-null  object
 1   维修ATA    55773 non-null  int64 
 2   问题描述     55773 non-null  object
 3   排故措施     30442 non-null  object
 4   运营人      55773 non-null  object
 5   机型       55773 non-null  object
 6   飞机序列号    55773 non-null  int64 
 7   机号       55773 non-null  object
 8   数据类型     55773 non-null  object
 9   拆卸部件件号   26445 non-null  object
 10  拆卸部件序列号  19823 non-null  object
 11  装上部件件号   26491 non-null  object
 12  装上部件序列号  19611 non-null  object
dtypes: int64(2), object(11)
memory usage: 6.0+ MB
